In [1]:
#predicting the median price of a house in Boston in mid 1970s given the data points about Boston which are crime rate,property taxes etc
from keras.datasets import boston_housing as BH
(train_data,train_targets),(test_data,test_targets) = BH.load_data()

Using TensorFlow backend.


In [2]:
print("The shape of training data set,training target,test data set and test targets")
print(train_data.shape)
print(train_targets.shape)
print(test_data.shape)
print(test_targets.shape)

The shape of training data set,training target,test data set and test targets
(404, 13)
(404,)
(102, 13)
(102,)


In [3]:
print("The house {} would go for {}".format(test_data[0],test_targets[0]))
#The range of values of each features are large.Hence we normalize.
#ex:for a house in boston with crime rate 1.55,ZN -0.483 ..... 13 columns,the price is 7.2k$

The house [ 18.0846   0.      18.1      0.       0.679    6.434  100.       1.8347
  24.     666.      20.2     27.25    29.05  ] would go for 7.2


In [4]:
#normalization of features vertically down the column
import numpy as np
mean = train_data.mean(axis = 0)
std = train_data.std(axis = 0)
train_data-=mean
train_data/=std
test_data-=mean
test_data/=std

In [5]:
from keras import models
from keras import layers

#building the model
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64,activation = "relu",input_shape = (13,)))
    model.add(layers.Dense(64,activation = "relu"))
    model.add(layers.Dense(1)) #output layer without activation since we want a linear prediction
    model.compile(optimizer = "rmsprop",loss = "mse",metrics = ["mae"])
    return model

In [6]:
#k fold validation
k = 4 #total number of folds
num_fold_samples = len(train_data)//k #no of samples per fold
num_epochs = 500
all_val_scores = []
all_mae_history = []
print(num_fold_samples)

101


In [7]:
import numpy as np
from sklearn.model_selection import KFold

kfold = KFold(k,False) #split into 4 groups without shuffling

#kfold.split() will give the array indices to use for training and validating
for (dataTuple,targetTuple) in zip(kfold.split(train_data),kfold.split(train_targets)): #splitting the data into train and validate using split
    (trainData,validateData) = dataTuple
    (trainLabel,validateLabel) = targetTuple
    model = build_model()
    historyObj = model.fit(train_data[trainData],train_targets[trainLabel],validation_data = (train_data[validateData],
                                                                   train_targets[validateLabel]),
                           batch_size = 1,
                           epochs = num_epochs,
                          verbose = 0)
    
    mae_history = historyObj.history["val_mean_absolute_error"]
    all_mae_history.append(mae_history)

In [8]:
#plotting the validation scores vs the epochs
import matplotlib.pyplot as plt
x = [x for x in range(10,num_epochs)]
y = [np.mean([x[i] for x in all_mae_history]) for i in range(10,num_epochs)]
plt.xlabel("epochs")
plt.ylabel("validation mae")
plt.plot(x,y)
plt.show()

<Figure size 640x480 with 1 Axes>

In [9]:
#hence from above graph we can tell that after 80 epochs the mae starts increasing and overfitting
#starts,hence optimal epoch = 80
#no need val_data as we have already done it
final_model = build_model()
final_model.fit(train_data,train_targets,epochs = num_epochs,batch_size = 16,verbose = 0)
test_mse,test_mae = final_model.evaluate(test_data,test_targets)

102/102 [==============================] - 0s 489us/step


In [10]:
print(test_mae)#we are still off by 2.63k$ for a predicted house

2.628593052134794
